# 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
# from langchain_huggingface import HuggingFaceEmbeddings

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', 500)

# 2. Preparing Data

In [3]:
full_df = pd.read_csv('../artikel/all_health_articles.csv')
# df.drop(columns=['description', 'content'], inplace=True)
df = full_df[['article_index', 'content']]
df.head()

,article_index,content
0,1,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi tingkat risiko batu ginjal biasa minum 8 sampai 10 gelas air putih hari jaga fungsi ginjal sehat 2 batas konsumsi garam asupan garam lebih tingkat tekan da...
1,2,jaga sehat mata mata peran aktivitas seharihari dari ajar interaksi lingkung sehat mata abai muncul dari iritasi ringan infeksi sakit serius risiko sebab buta jaga mata periksa rutin dokter biasa pola hidup sehat dukung fungsi lihat optimal iring tambah usia biasa seharihari kurang sehat fungsi mata turun cegah langkah sederhana jaga asupan nutrisi istirahat tatap layar lindung mata dari papar sinar lebih jaga kualitas lihat tahan lebih tips jaga sehat mata mata salah organ aktif hari lihat ...
2,3,sebab campak campak sakit tular sebab virus campak tampek sebar cepat dari orang orang mekanisme utama tular percik air liur droplet dari mulut hidung derita batuk bersin bicara tular sentuh benda kontaminasi virus sentuh hidung mulut mata puluh faktor sebab pengaruh vaksinasi campak mmr campak serang individu vaksin vaksin mmr measles mumps rubella bukti efektif cegah sakit anakanak terima vaksin lengkap milik risiko lebih infeksi milik kebal tubuh virus campak tampek virus usia bayi muda v...
3,4,campak campak rubella sakit tular sebab virus dari keluarga paramyxoviridae genus morbillivirus sakit serang salur napas timbul gejala sistemik demam batuk pilek sakit tenggorok mata merah ruam kulit campak sakit tular pindah dari orang orang udara percik air liur benda kontaminasi virus gejala campak gejala campak muncul inkubasi 7 sampai 14 hari gejala flu demam lemas batuk kering pilek sakit tenggorok diare muntah mata merah air sensitif cahaya bercak putih bintik koplik muncul mulut hari...
4,5,sehat gigi mulut sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran proses cerna makan masuk tinggal mulut bersih sisa kembang bakteri sehat mulut kait indah senyum napas segar tentu sehat tubuh akibat jaga sehat gigi mulut abai bersih mulut gigi picu ganggu sehat muncul bau mulut halitosis sisa makan busuk mulut akibat biasa rokok gigi lubang akibat tumpuk plak kikis email gigi radang gusi gingivitis atas kembang ...


# 3. Dense Encoding

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['content'])

In [5]:
print(embeddings)

[[-0.02351626  0.07242429 -0.02842112 ...  0.00293551 -0.07409749
  -0.04062068]
 [ 0.00615982  0.01363388 -0.01907845 ...  0.00788074 -0.07486614
  -0.05919098]
 [-0.01720827  0.01469457 -0.06229497 ... -0.05840543 -0.0243688
   0.02063797]
 ...
 [-0.00958528  0.01784758  0.01170867 ... -0.0413559  -0.04990955
   0.02661984]
 [-0.04443652  0.03816727 -0.0153152  ...  0.00447865  0.02433346
  -0.04069825]
 [ 0.02841109  0.0437499   0.02776248 ... -0.00588526 -0.02467242
  -0.0093273 ]]


In [6]:
embeddings.shape

(420, 384)

# 4. Building a Vector Index

In [7]:
index = faiss.IndexFlatL2(embeddings.shape[1])

In [8]:
index.is_trained

True

In [9]:
index.add(embeddings)

In [10]:
index.ntotal

420

# 5. Query and Retrieval

In [11]:
k = 10
example_query = ["gejala diabetes"]
query = model.encode(example_query)

In [12]:
distances, indices = index.search(query, k)
indices

array([[409, 413,  48,  85,  50, 352, 200,   0, 366,  99]])

In [13]:
print("Top 5 nearest neighbors:", indices)
print("Distances:", distances)

Top 5 nearest neighbors: [[409 413  48  85  50 352 200   0 366  99]]
Distances: [[0.8670476 0.9403507 0.9557108 0.9835525 0.989938  1.0441706 1.044924
  1.0752895 1.0770338 1.084089 ]]


In [32]:
raw_df = pd.read_csv('../artikel/raw_health_articles.csv')

In [33]:
len(raw_df)

420

In [34]:
results = raw_df[['article_index','title', 'content']].iloc[indices[0]]

In [35]:
results = pd.DataFrame(results)
results['similarity score'] = distances[0]

In [36]:
results

,article_index,title,content,similarity score
409,410,"Gejala Diabetes Tipe 1 pada Anak, Banyak Minum hingga Berat Badan Turun","KOMPAS.com -Diabetes tipe 1 pada anak masih sering terlambat dikenali, padahal gejalanya bisa terlihat dari kebiasaan sehari-hari. Jika orangtua lebih peka, kondisi ini bisa lebih cepat terdeteksi dan ditangani. ""Ketika didapati anak banyak makan, banyak minum, banyak kencing, berat badan turun drastis, yang tadinya tidakngompol,ngompollagi, apalagi anak mulai loyo, yang pertama harus dipikirkan adalah diabetes,"" jelas Prof. Dr. dr. Aman Bhakti Pulungan, Sp.A(K) Subsp. End., FAAP FRCPI (Hon....",0.867048
413,414,"Diabetes Tipe 1 pada Anak, Pentingnya Diagnosis Dini untuk Cegah Kematian","KOMPAS.com -Sebanyak 70 persen anak berusia di bawah 18 tahun di Indonesia telat terdiagnosis diabetes tipe 1. Oleh sebab itu, diabetes tipe 1 pada anak menjadi perhatian serius di Tanah Air. “Kita itu masih 70 persen pasien kita terdiagnosis telat dengan adanya ketoasidosis diabetik (KAD) ini. Ini kan bisa meninggal,” kata Prof. Dr. dr. Aman Bhakti Pulungan, Sp.A, Subsp. End., FAAP FRCPI (Hon.), dokter subspesialis Endokrinologi Anak di RS Cipto Mangunkusumo (RSCM), dilansir dariAntara, Kam...",0.940351
48,49,5 Faktor Risiko Diabetes yang Perlu Anda Ketahui,"Tahukah Biotizen, sekitar 9,1 juta orang di Indonesia diperkirakan menderita diabetes? Penyakit ini umumnya menyerang kelompok usia 55 hingga 74 tahun. Meski begitu, saat ini diabetes juga mulai banyak dialami oleh orang di usia muda, bahkan mulai dari 20-an hingga 40-an.\nApa itu diabetes dan siapa saja yang berisiko?\nDiabetes adalah kondisi kronis di mana kadar gula darah dalam tubuh terlalu tinggi akibat gangguan produksi atau fungsi insulin. Sayangnya, banyak orang tidak menyadari bahwa...",0.955711
85,86,Gejala Diabetes Melitus yang Harus Kamu Ketahui!,"Diabetes mellitus adalah suatu kondisi yang ditandai dengan tingginya kadar gula (glukosa) dalam darah secara terus-menerus. Hal ini terjadi karena tubuh tidak dapat memproduksi insulin yang cukup atau tidak dapat menggunakan insulin secara efektif. Insulin adalah hormon yang diperlukan untuk mengubah glukosa menjadi energi. Ketika tubuh tidak menghasilkan cukup insulin atau sel-sel tubuh tidak merespons insulin secara efektif, kadar glukosa dalam darah meningkat, yang dikenal sebagai hiperg...",0.983553
50,51,Bahaya Diabetes Mellitus dan Faktor Risiko yang Meningkatkan Kemungkinannya,"Diabetes Mellitus (DM) merupakan salah satu masalah kesehatan global yang sangat serius, dengan dampak yang dapat mengancam kualitas hidup penderitanya. Penyakit ini tidak hanya menyebabkan komplikasi jangka panjang yang bisa mempengaruhi banyak sistem tubuh, tetapi juga bisa berujung pada komplikasi akut yang membahayakan nyawa. Ironisnya, banyak orang baru menyadari bahaya DM ketika komplikasi sudah muncul, hal ini disebabkan oleh kurangnya pemahaman mengenai penyakit ini. Kurangnya penget...",0.989938
352,353,Efek Samping Metformin pada Pengguna Diabetes Tipe 2 dan Tips Pencegahannya,"Efek samping metformin dapat dialami oleh sebagian pengguna obat diabetes tipe 2, mulai dari gejala ringan hingga kondisi serius yang memerlukan penanganan segera. Memahami efek samping metformin penting agar Anda bisa mengenalinya lebih awal dan tahu apa yang harus dilakukan bila muncul keluhan.\r\nMetformin adalah obat yang dapat membantu menurunkan kadar gula darah, terutama pada penderita diabetes tipe 2, dengan cara meningkatkan respons tubuh terhadap insulin. Meski efektif dan sering m...",1.044171
200,201,"Hari Diabetes Sedunia, Diabetasol Gelar Edukasi Kesehatan","Berita Kesehatan\nHari Diabetes Sedunia, Diabetasol Gelar Edukasi Kesehatan\nDimas Laksono, 10 Nov 2022\nDitinjau Oleh Tim Medis Klikdokter\nBagikan\nMemperingati Hari Diabetes Sedunia, Diabetasol menggelar rangkaian acara talk show di 12 daerah secara online. Acara ini juga berlangsung di 5 negara Asia termasuk Indonesia.\nMemperingati Hari Diabetes Sedunia pada 14 Novemb

# 6 Coba

In [66]:
embeddings.embed_query("buah")

AttributeError: 'numpy.ndarray' object has no attribute 'embed_query'